## Calculating similarity score using Jaro Winkler distance.

In [6]:
!pip install docx2txt

In [7]:
!pip install strsimpy

In [8]:
# importing all necessary libraries
import pandas as pd
import numpy as np
import docx2txt
import json, collections
from pandas.io.json import json_normalize
import os

In [9]:
!pip install docx2csv

In [10]:
from docx2csv import extract_tables, extract
tables = extract_tables('/content/lists.docx')

In [11]:
import pandas as pd
from docx import Document
document = Document("/content/lists.docx")

In [12]:
def read_docx_table(document,table_num):
  table = document.tables[table_num-1]
  data = [[cell.text for cell in row.cells] for row in table.rows]
  df = pd.DataFrame(data)
  return df

In [13]:
table_num=1
df = read_docx_table(document,table_num)
df.head()

,0,1
0,P1,Analyze a complex computing problem and to app...
1,P2,"Design, implement, and evaluate a computing-ba..."
2,P3,Communicate effectively in a variety of profes...
3,P4,Recognize professional responsibilities and ma...
4,P5,Function effectively as a member or leader of ...


In [14]:
q1 = df.copy()

In [15]:
table_num=2
df1 = read_docx_table(document,table_num)
df1.head()

,0,1
0,C1,Explain how AI impacts society
1,C2,Apply AI methods to perform practical tasks
2,C3,Synthesize a simple AI system
3,C4,Evaluate different AI approaches to solve a pr...
4,C5,Explain the basic principles of the data minin...


In [16]:
P1 = ["Analyze a complex computing problem and to apply principles of computing and other relevant disciplines to identify solutions."]
p1 = df1.copy()
p1.head()

,0,1
0,C1,Explain how AI impacts society
1,C2,Apply AI methods to perform practical tasks
2,C3,Synthesize a simple AI system
3,C4,Evaluate different AI approaches to solve a pr...
4,C5,Explain the basic principles of the data minin...


In [17]:
df1 = pd.DataFrame({
    "name": ["mahesh", "suresh"]
})

df2 = pd.DataFrame({
    "name": ["mahesh", "surendra", "shrivatsa", "suresh", "maheshwari"]
})

df = pd.MultiIndex.from_product(
    [df1["name"], df2["name"]], names=["col1", "col2"]
).to_frame(index=False)


In [18]:
d = pd.MultiIndex.from_product(
    [q1[1], p1[1]], names=["col1", "col2"]
).to_frame(index=False)

In [19]:
d.head()

,col1,col2
0,Analyze a complex computing problem and to app...,Explain how AI impacts society
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...


In [20]:
d.shape

(786, 2)

In [21]:
d.head(133)

,col1,col2
0,Analyze a complex computing problem and to app...,Explain how AI impacts society
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...
...,...,...
128,Analyze a complex computing problem and to app...,Define the computing requirements appropriate ...
129,Analyze a complex computing problem and to app...,Apply knowledge of computer science theory and...
130,Analyze a complex computing problem and to app...,Develop a computer science application.
131,"Design, implement, and evaluate a computing-ba...",Explain how AI impacts society


In [22]:
from strsimpy.jaro_winkler import JaroWinkler
jarowinkler = JaroWinkler()
d["jarowinkler_sim"] = [jarowinkler.similarity(i,j) for i,j in zip(d["col1"],d["col2"])]

In [23]:
d.drop(["col1", "col2"], axis=1, inplace=True)
d.head()

,jarowinkler_sim
0,0.530582
1,0.602081
2,0.528113
3,0.608833
4,0.632987


In [25]:
d.tail()

,jarowinkler_sim
781,0.659523
782,0.676995
783,0.722682
784,0.765312
785,0.588645


In [26]:
def split_dataframe(df, chunk_size = 131): 
    chunks = list()
    num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size else 0)
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [27]:
d_split2 = split_dataframe(d, chunk_size=131)

In [28]:
d_split2

[     jarowinkler_sim
 0           0.530582
 1           0.602081
 2           0.528113
 3           0.608833
 4           0.632987
 ..               ...
 126         0.675743
 127         0.711717
 128         0.691077
 129         0.719781
 130         0.588116
 
 [131 rows x 1 columns],      jarowinkler_sim
 131         0.481270
 132         0.521575
 133         0.484241
 134         0.565839
 135         0.590232
 ..               ...
 257         0.649035
 258         0.683164
 259         0.672855
 260         0.650728
 261         0.543645
 
 [131 rows x 1 columns],      jarowinkler_sim
 262         0.553125
 263         0.615051
 264         0.552971
 265         0.661161
 266         0.674219
 ..               ...
 388         0.646074
 389         0.597363
 390         0.653897
 391         0.662616
 392         0.659668
 
 [131 rows x 1 columns],      jarowinkler_sim
 393         0.489364
 394         0.568971
 395         0.498177
 396         0.603898
 397         0.61285

In [29]:
d_split2[0].head()

,jarowinkler_sim
0,0.530582
1,0.602081
2,0.528113
3,0.608833
4,0.632987


In [30]:
d_split2[1].head()

,jarowinkler_sim
131,0.481270
132,0.521575
133,0.484241
134,0.565839
135,0.590232


In [31]:
d_split2[2].head()

,jarowinkler_sim
262,0.553125
263,0.615051
264,0.552971
265,0.661161
266,0.674219


In [32]:
d_split2[3].head()

,jarowinkler_sim
393,0.489364
394,0.568971
395,0.498177
396,0.603898
397,0.612855


In [33]:
d_split2[4].tail()

,jarowinkler_sim
650,0.641028
651,0.733730
652,0.706088
653,0.672622
654,0.565620


In [34]:
d_split2[5].head()

,jarowinkler_sim
655,0.543810
656,0.645395
657,0.532608
658,0.646662
659,0.632540


In [35]:
d_split2[0].rename(columns = {'jarowinkler_sim':'P1'}, inplace = True)
d_split2[0].head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,P1
0,0.530582
1,0.602081
2,0.528113
3,0.608833
4,0.632987


In [36]:
d_split2[1].rename(columns = {'jarowinkler_sim':'P2'}, inplace = True)
d_split2[2].rename(columns = {'jarowinkler_sim':'P3'}, inplace = True)
d_split2[3].rename(columns = {'jarowinkler_sim':'P4'}, inplace = True)
d_split2[4].rename(columns = {'jarowinkler_sim':'P5'}, inplace = True)
d_split2[5].rename(columns = {'jarowinkler_sim':'P6'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
d_split2[5].reset_index(drop=True, inplace=True)
d_split2[5].head()

,P6
0,0.543810
1,0.645395
2,0.532608
3,0.646662
4,0.632540


In [38]:
d_split2[1].reset_index(drop=True, inplace=True)
d_split2[2].reset_index(drop=True, inplace=True)
d_split2[3].reset_index(drop=True, inplace=True)
d_split2[4].reset_index(drop=True, inplace=True)

In [39]:
dd = pd.concat([d_split2[0], d_split2[1], d_split2[2], d_split2[3], d_split2[4], d_split2[5]], axis=1)
dd.head()

,P1,P2,P3,P4,P5,P6
0,0.530582,0.481270,0.553125,0.489364,0.474879,0.543810
1,0.602081,0.521575,0.615051,0.568971,0.553719,0.645395
2,0.528113,0.484241,0.552971,0.498177,0.471189,0.532608
3,0.608833,0.565839,0.661161,0.603898,0.607900,0.646662
4,0.632987,0.590232,0.674219,0.612855,0.617369,0.632540


In [40]:
dd.tail()

,P1,P2,P3,P4,P5,P6
126,0.675743,0.649035,0.646074,0.688629,0.641028,0.659523
127,0.711717,0.683164,0.597363,0.696824,0.733730,0.676995
128,0.691077,0.672855,0.653897,0.687895,0.706088,0.722682
129,0.719781,0.650728,0.662616,0.689211,0.672622,0.765312
130,0.588116,0.543645,0.659668,0.595887,0.565620,0.588645


In [41]:
df2 = pd.concat([p1[0], dd], axis=1)
df2.head()

,0,P1,P2,P3,P4,P5,P6
0,C1,0.530582,0.481270,0.553125,0.489364,0.474879,0.543810
1,C2,0.602081,0.521575,0.615051,0.568971,0.553719,0.645395
2,C3,0.528113,0.484241,0.552971,0.498177,0.471189,0.532608
3,C4,0.608833,0.565839,0.661161,0.603898,0.607900,0.646662
4,C5,0.632987,0.590232,0.674219,0.612855,0.617369,0.632540


In [42]:
df2.set_index(0, inplace=True)
df2.head()

,P1,P2,P3,P4,P5,P6
0,,,,,,
C1,0.530582,0.481270,0.553125,0.489364,0.474879,0.543810
C2,0.602081,0.521575,0.615051,0.568971,0.553719,0.645395
C3,0.528113,0.484241,0.552971,0.498177,0.471189,0.532608
C4,0.608833,0.565839,0.661161,0.603898,0.607900,0.646662
C5,0.632987,0.590232,0.674219,0.612855,0.617369,0.632540


In [43]:
df2.tail()

,P1,P2,P3,P4,P5,P6
0,,,,,,
C127,0.675743,0.649035,0.646074,0.688629,0.641028,0.659523
C128,0.711717,0.683164,0.597363,0.696824,0.733730,0.676995
C129,0.691077,0.672855,0.653897,0.687895,0.706088,0.722682
C130,0.719781,0.650728,0.662616,0.689211,0.672622,0.765312
C131,0.588116,0.543645,0.659668,0.595887,0.565620,0.588645
